In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

def mainBareBones():
    # Load the dataset
    file_path = "Loan Prediction.csv"  # Replace with the path to your dataset
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    
    # Fill missing values for categorical and numerical columns
    for col in data.columns:
        if data[col].dtype == 'object':  # Categorical columns
            data[col] = data[col].fillna(data[col].mode()[0])
        else:  # Numerical columns
            data[col] = data[col].fillna(data[col].median())
    
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Convert categorical variables to numeric
    print("Converting categorical variables to numeric with one-hot encoding...")
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)
    print(f"Data shape after encoding: {data.shape}")

    # Feature-target split
    print("\nSeparating features and target variable...")
    X = data.drop(columns=['Risk_Flag', 'Id'])  # Dropping Id and target column
    y = data['Risk_Flag']
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Train-test split
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Train Random Forest model
    print("\nTraining the Random Forest classifier...")
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

if __name__ == "__main__":
    mainBareBones()


Loading dataset...
Dataset loaded successfully! Shape: (252000, 13)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0
Converting categorical variables to numeric with one-hot encoding...
Data shape after encoding: (252000, 405)

Separating features and target variable...
Features shape: (252000, 403), Target shape: (252000,)

Splitting data into training and testing sets...
Training set shape: (201600, 403), Testing set shape: (50400, 403)

Training the Random Forest classifier...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy: 89.85%

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.95      0.94     44147
           1       0.60      0.54      0.57      6253

    accuracy                           0.90     50400
   macro avg       0.77      0.74      0.75     50400
weighted avg       0.89      0.90      0.90     50400



In [2]:
import cupy as cp  # Import CuPy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier  # Use Random Forest
import numpy as np  # Import NumPy
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

def mainCuPY():
    # Load the dataset
    file_path = "Loan Prediction.csv"  # Replace with the path to your dataset
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    
    # Fill missing values for categorical and numerical columns
    for col in data.columns:
        if data[col].dtype == 'object':  # Categorical columns
            data[col] = data[col].fillna(data[col].mode()[0])
        else:  # Numerical columns
            data[col] = data[col].fillna(data[col].median())
    
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Convert categorical variables to numeric
    print("Converting categorical variables to numeric with one-hot encoding...")
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)
    print(f"Data shape after encoding: {data.shape}")

    # Feature-target split
    print("\nSeparating features and target variable...")
    X = data.drop(columns=['Risk_Flag', 'Id'])  # Dropping Id and target column
    y = data['Risk_Flag']
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Convert to NumPy arrays before converting to CuPy arrays
    print("\nConverting DataFrame to arrays...")
    X = X.values  # DataFrame to NumPy
    y = y.values

    # Ensure the data contains no invalid values (inf, -inf, NaN)
    print("\nCleaning invalid values in the dataset...")
    X = pd.DataFrame(X).replace([np.inf, -np.inf], np.nan).dropna().values
    
    # Ensure all columns in X are numeric and cast to float
    if not np.issubdtype(X.dtype, np.number):
        X = pd.DataFrame(X).apply(pd.to_numeric, errors='coerce').fillna(0).values
    
    # Explicitly cast X and y to float32
    X = X.astype(np.float32)  
    y = y.astype(np.float32)  
    
    # Convert NumPy arrays to CuPy arrays
    print("\nTransferring data to GPU...")
    X = cp.asarray(X)
    y = cp.asarray(y)
    print("Data successfully transferred to GPU!")

    # Train-test split
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(cp.asnumpy(X), cp.asnumpy(y), test_size=0.2, random_state=42)
    X_train = cp.asarray(X_train)
    X_test = cp.asarray(X_test)
    y_train = cp.asarray(y_train)
    y_test = cp.asarray(y_test)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Train Random Forest model
    print("\nTraining the Random Forest classifier...")
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(cp.asnumpy(X_train), cp.asnumpy(y_train))  # Use `.asnumpy()` to transfer data back to CPU for sklearn
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = cp.asarray(model.predict(cp.asnumpy(X_test)))  # Use `.asnumpy()` to transfer data to CPU, then back to GPU

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(cp.asnumpy(y_test), cp.asnumpy(y_pred))  # Use `.asnumpy()` for sklearn functions
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(cp.asnumpy(y_test), cp.asnumpy(y_pred), zero_division=1))

if __name__ == "__main__":
    mainCuPY()

Loading dataset...
Dataset loaded successfully! Shape: (252000, 13)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0
Converting categorical variables to numeric with one-hot encoding...
Data shape after encoding: (252000, 405)

Separating features and target variable...
Features shape: (252000, 403), Target shape: (252000,)

Converting DataFrame to arrays...

Cleaning invalid values in the dataset...

Transferring data to GPU...
Data successfully transferred to GPU!

Splitting data into training and testing sets...
Training set shape: (201600, 403), Testing set shape: (50400, 403)

Training the Random Forest classifier...



KeyboardInterrupt



In [3]:
import pandas as pd
import numpy as np
from sklearnex import patch_sklearn 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# Apply the patch to accelerate sklearn
patch_sklearn()

def mainIntelex():
    # Load the dataset
    file_path = "Loan Prediction.csv"  # Replace with the path to your dataset
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    
    # Fill missing values for categorical and numerical columns
    for col in data.columns:
        if data[col].dtype == 'object':  # Categorical columns
            data[col] = data[col].fillna(data[col].mode()[0])
        else:  # Numerical columns
            data[col] = data[col].fillna(data[col].median())
    
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Convert categorical variables to numeric
    print("Converting categorical variables to numeric with one-hot encoding...")
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)
    print(f"Data shape after encoding: {data.shape}")

    # Feature-target split
    print("\nSeparating features and target variable...")
    X = data.drop(columns=['Risk_Flag', 'Id'])  # Dropping Id and target column
    y = data['Risk_Flag']
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Train-test split
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Train Random Forest model
    print("\nTraining the Random Forest classifier...")
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

if __name__ == "__main__":
    mainIntelex()


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Loading dataset...
Dataset loaded successfully! Shape: (252000, 13)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0
Converting categorical variables to numeric with one-hot encoding...
Data shape after encoding: (252000, 405)

Separating features and target variable...
Features shape: (252000, 403), Target shape: (252000,)

Splitting data into training and testing sets...
Training set shape: (201600, 403), Testing set shape: (50400, 403)

Training the Random Forest classifier...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy: 89.85%

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.95      0.94     44147
           1       0.60      0.54      0.57      6253

    accuracy                           0.90     50400
   macro avg       0.77      0.74      0.75     50400
weighted avg       0.89      0.90      0.90     50400

